## Hors ligne RL

In [1]:
# HIDDEN
import gym
import numpy as np

In [2]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Est-ce que c'est réaliste ?

- Jusqu'à présent, nous avons construit une simulation du comportement de l'utilisateur
- Dans certaines applications, nous pourrions être en mesure de construire des simulations précises :
  - simulations de physique (par exemple, les robots)
  - jeux
  - simulations économiques/financières ?
- Cependant, pour le comportement de l'utilisateur, c'est difficile

#### Est-ce que c'est réaliste ? 

- Le mieux serait de déployer RL en direct, mais ce n'est pas pratique
- Une autre possibilité : apprendre à partir des données des utilisateurs ?
- Nous pouvons le faire avec **l'apprentissage par renforcement en ligne**

#### RL hors ligne

- Qu'est-ce que le RL hors ligne ?
- Rappelle-toi notre boucle RL :

[](img/RL-loop-3.png)

#### RL hors ligne

- Dans la RL hors ligne, nous n'avons pas d'environnement avec lequel interagir dans une boucle de rétroaction :

[](img/offline-RL-loop.png)

Ces données historiques ont été générées par une/des autre(s) politique(s) inconnue(s).

Notes :

Peut être généré par des utilisateurs réels, ou par une source différente (aléatoire, ou agent RL !)

#### Le défi de la RL hors ligne

- On ne peut pas répondre aux questions "et si"
- Nous ne pouvons voir que les résultats des actions tentées dans l'ensemble de données

Remarques :

Peut-être que cela nous fait apprécier à quel point il est précieux/important d'avoir réellement un env à disposition, ce que nous avons eu pendant tout le reste du cours. Cela nous permet d'essayer n'importe quoi sans aucun coût, sauf le coût de calcul (en supposant qu'il s'agit d'un simulateur et non d'un environnement du monde réel) 

In [3]:
# # HIDDEN

# # generate the offline dataset
# env_config = {
#     "num_candidates" : 2,
#     "alpha"          : 0.5,
#     "seed"           : 42
# }

# from ray.rllib.algorithms.ppo import PPO, PPOConfig

# ppo_config = (
#     PPOConfig()\
#     .framework("torch")\
#     # need to set num_rollout_workers=1 for now per https://github.com/ray-project/ray/issues/25696
#     .rollouts(create_env_on_local_worker=True, num_rollout_workers=1)\
#     .debugging(seed=0, log_level="ERROR")\
#     .training(model={"fcnet_hiddens" : [64, 64]}, lr=0.001)\
#     # .environment(env_config=env_config)\
#     .offline_data(output="data/recommender2")
# )

# from envs_04 import BasicRecommenderWithHistory

# ppo_history = ppo_config.build(env="CartPole-v1")

# rewards_history = []
# for i in range(25):
#     result = ppo_history.train()
#     rewards_history.append(result["episode_reward_mean"])
    
# ppo_history.evaluate(duration_fn=1000)["evaluation"]["episode_reward_mean"]

#### Jeu de données Recommender

- Explorons un ensemble de données hors ligne dont nous pouvons tirer des enseignements.
- Nous aurons besoin d'un peu de code pour lire tous les objets JSON du fichier :

In [4]:
import json

json_dataset_file = "data/recommender_offline.json"

rollouts = []
with open(json_dataset_file, "r") as f:
    for line in f:
        data = json.loads(line)
        rollouts.append(data)

In [5]:
len(rollouts)

50

Nous avons 50 "déploiements" de données.

Notes :

Le fichier est dans le format à partir duquel RLlib apprend.

#### Jeu de données Recommender

Chaque rollout est un dict contenant des informations sur le pas de temps :

In [6]:
from ray.rllib.utils.compression import unpack, pack

obs = unpack(rollouts[0]["obs"])
obs.shape

(200, 2)

- Nous avons 200 étapes temporelles de données dans chaque déploiement
- Examinons d'abord les observations

Notes :

Ce nombre 200 est défini par le paramètre de configuration de l'algorithme "rollout_fragment_length".

#### Jeu de données Recommender

Voici les 3 premières observations :

In [7]:
obs[:3]

array([[0.6545137 , 0.29728338],
       [0.5238871 , 0.5144319 ],
       [0.6741674 , 0.10163702]], dtype=float32)


Nous pouvons voir que `num_candidats` a été fixé à 2

#### Jeu de données Recommender

Nous pouvons également examiner les 3 premières actions, les récompenses et les dons :

In [8]:
rollouts[0]["actions"][:3]

[0, 0, 1]

In [9]:
rollouts[0]["rewards"][:3]

[0.6545137166976929, 0.3524414300918579, 0.05838315561413765]

In [10]:
rollouts[0]["dones"][:3]

[False, False, False]

Notes :

Donc, l'agent a d'abord vu l'observation [0.65, 0.297] de la diapositive précédente, puis il a fait l'action 0, a obtenu une récompense de 0.65, et l'épisode n'a pas été fait.

Il y a plus d'informations stockées dans l'ensemble de données que ce qui précède, mais ce sont les points clés.

#### Apprentissage supervisé

- Attends, c'est un ensemble de données... ne pouvons-nous pas simplement faire de l'apprentissage supervisé pour faire correspondre les états aux actions ? 🤔
- Oui, nous le pouvons, et cela viserait à récupérer la politique qui a généré l'ensemble de données (_apprentissage par imitation_).
- Mais il est en fait possible de faire _mieux_... et c'est notre objectif avec le RL hors ligne.

In [11]:
# TODO
# maybe going overboard here, but for a synthetic example we could actually show it does better than the policy that generated the data,
# since we generated the data ourselves

#### Formation RL hors ligne

- Beaucoup d'informations sur le RL hors ligne avec RLlib [ici] (https://docs.ray.io/en/latest/rllib/rllib-offline.html).
- Tout d'abord, nous avons besoin d'un algorithme.
- Pour le RL hors ligne, nous ne pouvons pas utiliser `PPO`.
- Nous utiliserons l'algorithme `MARWIL` qui est inclus dans RLlib.

In [12]:
from ray.rllib.algorithms.marwil import MARWIL, MARWILConfig

#### Formation RL hors ligne

Ensuite, nous créons la config, en commençant par `MARWILConfig` au lieu de `PPOConfig`

In [13]:
# This is the same as before
offline_config = ( 
    MARWILConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True)\
    .debugging(seed=0, log_level="ERROR")\
    .training(model={"fcnet_hiddens" : [64, 64]})\
)

# This is new for offline RL
num_candidates = 2
offline_config = offline_config.environment(
    observation_space = gym.spaces.Box(low=0, high=1, shape=(num_candidates,)), 
    action_space = gym.spaces.Discrete(num_candidates),
).offline_data(
    input_ = [json_dataset_file],
)

Notes :

- Les éléments de configuration du haut devraient te sembler familiers. Sur la deuxième moitié, les choses sont un peu différentes :
  - Nous devons lui donner le chemin d'accès au fichier de l'ensemble de données
  - Comme il n'y a pas d'env, nous devons spécifier manuellement les espaces d'observation et d'action
- Nous n'avons pas de config d'environnement car il n'y a pas d'environnement !

#### Formation

In [14]:
marwil = offline_config.build()

In [15]:
for i in range(50):
    marwil.train()

#### Évaluation

_Comment évaluer sans simulateur ?

- C'est ce qu'on appelle l'estimation hors politique.
- C'est une technique plus sophistiquée qui sort du cadre de ce cours.
- Voir la documentation de RLlib [ici] (https://docs.ray.io/en/latest/rllib/rllib-offline.html).
- Ce que nous allons faire, c'est évaluer avec notre simulateur, puisque nous en avons un.

Notes :

Dans un scénario réaliste de RL hors ligne, tu n'as pas de simulateur à ta disposition. Tu dois alors utiliser des techniques d'évaluation plus complexes appelées estimation hors ligne. Tu auras des ensembles de données d'entraînement/de test comme dans l'apprentissage supervisé 

Pour nos besoins, puisque nous avons un simulateur, nous l'utiliserons pour l'évaluation. RLlib a en fait une option pour permettre cela, car c'est utile pour le débogage et autres. Nous verrons cela sur la prochaine diapositive.

#### Évaluation avec notre simulateur

Nous pouvons évaluer l'algorithme à l'aide de notre simulateur env.

In [16]:
# HIDDEN
from envs_04 import BasicRecommender

env_config = {
    "num_candidates" : 2,
    "alpha"          : 0.5,
    "seed"           : 42
}

In [17]:
env = BasicRecommender(env_config)

def get_episode_reward(env, algo):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = algo.compute_single_action(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward
    return total_reward

- Ci-dessus : configure une fonction qui exécute un épisode avec le simulateur.
- Ci-dessous : exécute cette fonction pour 100 épisodes afin d'obtenir la moyenne.

In [18]:
np.mean([get_episode_reward(env, marwil) for i in range(100)])

25.537665635536726

Cela semble faire à peu près la même chose que le hasard à nouveau (25,5).

#### RLlib pour l'évaluation sur simulateur

RLlib propose également cette fonctionnalité :

In [31]:
offline_config = offline_config.evaluation(
    off_policy_estimation_methods={},
    evaluation_config={
        "input": "sampler",
        "env": BasicRecommender,
        "env_config" : env_config
    }
)

In [ ]:
# HIDDEN
#marwil.stop()

Si nous avions mis en place la configuration de cette façon et formé :

In [32]:
marwil = offline_config.build()

In [33]:
for i in range(10):
    marwil.train()

In [30]:
marwil.evaluate()["evaluation"]["episode_reward_mean"]

25.3457144503546

Ici, nous voyons à nouveau un résultat similaire.

#### Appliquons ce que nous avons appris !

## Exemple de RL hors ligne
<!-- multiple choice -->

- [ ] Apprendre à une IA à jouer aux échecs en la faisant jouer contre d'autres IA à plusieurs reprises.
- [ ] Apprendre à une IA à jouer aux échecs en se basant sur les parties passées de joueurs d'échecs professionnels.

## Les données hors ligne t'aident-elles ?
<!-- multiple choice -->

Imagine que tu as un simulateur qui représente parfaitement ton environnement ; par exemple, tu entraînes peut-être une IA à jouer à un jeu solo comme [Atari Breakout](https://en.wikipedia.org/wiki/Breakout_(video_game)). En plus du simulateur, tu as aussi des données hors ligne à ta disposition. Bien que cela ne soit pas abordé dans les diapositives ci-dessus, il est possible de combiner un simulateur avec des données hors ligne pour former conjointement un agent à l'aide de RL (voir [ici](https://docs.ray.io/en/latest/rllib/rllib-offline.html#mixing-simulation-and-offline-data)). Si tu as déjà un simulateur parfait, les données hors ligne pourraient-elles apporter une utilité supplémentaire, si elles sont combinées avec le simulateur pendant la formation ?

#### Meilleure politique après l'entraînement

Choisis l'affirmation correcte concernant la recherche de la meilleure politique après une formation illimitée. Tu peux supposer que tu as un algorithme RL "parfait", ce qui signifie qu'il peut représenter n'importe quelle politique et est capable d'optimiser n'importe quelle fonction.

- [Avec un algorithme RL "parfait" et un temps de calcul suffisant, il n'y a aucun avantage à utiliser les données historiques car le simulateur contient tout ce qu'il y a à savoir sur l'environnement.
- [ ] Avec un algorithme RL "parfait" et un temps de calcul illimité, les données historiques peuvent t'aider à trouver une meilleure politique qu'en utilisant uniquement le simulateur. | Avec un algorithme RL "parfait", tu devrais finir par trouver la politique optimale. Cela revient à avoir des données infinies, un temps de calcul infini, un modèle arbitrairement complexe et une méthode d'optimisation parfaite.

#### Vitesse d'entraînement

Choisis l'affirmation correcte concernant le fait de trouver la meilleure politique après un peu d'entraînement. Tu peux supposer que tu as un algorithme RL "parfait", c'est-à-dire qu'il peut représenter n'importe quelle politique et est capable d'optimiser n'importe quelle fonction.

- [Avec un algorithme RL "parfait" mais seulement un peu de temps de calcul, il n'y a aucun avantage à utiliser les données historiques car le simulateur contient tout ce qu'il y a à savoir sur l'environnement. | Et si les données historiques étaient générées à l'aide de la politique *optimale* ?
- [Avec un algorithme RL "parfait" mais un temps de calcul limité, les données historiques peuvent t'aider à trouver une meilleure politique qu'en utilisant uniquement le simulateur. | Si les données historiques ont été générées à l'aide d'une très bonne politique, tu pourrais en tirer des leçons rapidement.

## Politique de données historiques
<!-- multiple choice -->

La RL hors ligne s'appuie sur des données générées par une certaine politique qui interagit avec l'environnement. Lequel des éléments suivants est **NON** une propriété souhaitable de cet ensemble de données / politique historique ?
 
- [x] L'environnement et la politique historique sont tous deux déterministes. | Dans ce cas, nous n'explorerions qu'une seule trajectoire à travers l'environnement 
- [ ] L'ensemble de données contient un grand nombre d'épisodes.
- [ ] La politique historique explore une variété d'états dans l'environnement.
- [ ] La politique historique obtient une récompense élevée dans certains épisodes.   

## RL hors ligne pour Cartpole
<!-- coding exercise -->

Dans cet exercice, nous allons nous attaquer au célèbre [problème de référence Cartpole] (https://www.gymlibrary.ml/environments/classic_control/cart_pole/) qui est fourni avec la bibliothèque `gym`. Le but est d'empêcher le pendule inversé de tomber en appliquant une force, à gauche ou à droite, à chaque pas de temps 

Nous entraînerons l'agent en utilisant des données hors ligne contenues dans un fichier `cartpolev1_offline.json` qui est lu par le code.

Nous _évaluerons_ l'agent en utilisant le vrai simulateur Cartpole. (Encore une fois, dans la réalité, si nous utilisions RL hors ligne, nous n'aurions probablement pas accès au vrai simulateur, mais nous l'incluons ici afin de pouvoir faire une évaluation de terrain de notre agent)

Remplis le code manquant. Ensuite, exécute le code et réponds à la question à choix multiple ci-dessous.

In [ ]:
# EXERCISE
import gym
import numpy as np
import matplotlib.pyplot as plt

from ray.rllib.algorithms.marwil import MARWIL, MARWILConfig

offline_trainer_config = {
    # These should look familiar:
    "framework"             : "torch",
    "create_env_on_driver"  : True,
    "seed"                  : 0,
    "model"                 : {
        "fcnet_hiddens"     : [64, 64]
    },
    
    # These are new for offline RL:
    ____: ["data/cartpolev1_offline.json"],
    "observation_space": gym.spaces.Box(low=____, 
                                        high=np.array([4.8,  np.inf,  0.42,  np.inf])),
    "action_space": gym.spaces.Discrete(2),
    "input_evaluation" : ["simulation", "is", "wis"],
    "env" : "CartPole-v1" # for evaluation only
}

algo = ...

# Training (and storing results)
results_off = []
results_sim = []
for i in range(200):
    r = algo.____()
    results_off.append(r["off_policy_estimator"]["wis"]['V_gain_est'])
    results_sim.append(r["episode_reward_mean"])

plt.plot(results_sim);
plt.xlabel('iterations') 
plt.ylabel('simulator reward') 

In [30]:
# SOLUTION
import gym
import numpy as np
import matplotlib.pyplot as plt
from ray.rllib.algorithms.marwil import MARWIL, MARWILConfig
from ray.rllib.algorithms.crr import CRR, CRRConfig

# This is the same as before
offline_config = ( 
    CRRConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True)\
    .debugging(seed=0, log_level="ERROR")\
    .training(model={"fcnet_hiddens" : [32, 32]})\
)
# This is new for offline RL
offline_config = offline_config.environment(
    observation_space = gym.spaces.Box(low=np.array([-4.8, -np.inf, -0.42, -np.inf]), 
                                        high=np.array([4.8,  np.inf,  0.42,  np.inf])),
    action_space = gym.spaces.Discrete(2),
    env = "CartPole-v1" # for evaluation only
).offline_data(
    input_ = ["data/cartpolev1_offline.json"]
).evaluation(
    off_policy_estimation_methods={'simulation': {'type': 'simulation'}}
)

algo = offline_config.build()

# Training (and storing results)
results_off = []
results_sim = []
for i in range(10):
    print(i)
    r = algo.train()
    results_off.append(r[""]["wis"]['v_new_mean'])
    results_sim.append(r["episode_reward_mean"])

plt.plot(results_sim);
plt.xlabel('iterations') 
plt.ylabel('simulator reward') 

ValueError: not enough values to unpack (expected 2, got 1)

In [31]:
# TODO
# see the tuned examples here: https://docs.ray.io/en/master/rllib/rllib-algorithms.html#offline

In [ ]:
# TODO:
# - ALSO: this might be a great example to try supervised learning and show why it doesn't work...???
#   - in some ways this is a way better example than frozen lake... because there IS a short-term reward, it's just not what you should look at.
#   - with frozen lake there is no short-term reward, so RL seems "obvious"
# Or, that could go in the offline RL section, since we already have a data file there and could do SL directly on it
# Yes, that seems cool.
# could use scipy/numpy to just do the normal equations if we want to avoid adding a dependency on sklearn 

# Supervised learning only gives you imitation learning, which can only get as good as the policy that generated the data